In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker, relationship, join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from statsmodels.regression.quantile_regression import QuantReg

from PIL import Image
from copy import copy
from aquabyte.visualize import _normalize_world_keypoints

from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D
import pickle
from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager

import sys
sys.path.append('/root/alok/repos/cv_research/alok/biomass_estimation/production_data_analysis_v3')
from template_matching import enhance, find_matches_and_homography, adjust_keypoints


pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 50)

<h1> Download Data </h1>

In [ ]:
# AWS credentials
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")
s3_access_utils = S3AccessUtils('/root/data')


rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS'])))
query = """
    select * from research.fish_metadata a left join keypoint_annotations b
    on a.left_url = b.left_image_url 
    where b.keypoints is not null and b.is_qa = false
    limit 1;
"""
df = rds_access_utils.extract_from_database(query)


In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.keypoints and 'rightCrop' in row.keypoints:
        return pixel2world(row.keypoints['leftCrop'], row.keypoints['rightCrop'], row.camera_metadata)
    else:
        return None
    
df['world_keypoints'] = df.apply(
    lambda x: get_world_keypoints(x), axis=1
)

In [ ]:
for idx, row in df.iterrows():
    keypoints = row.keypoints
    if 'leftCrop' in keypoints and 'rightCrop' in keypoints:
        if (keypoints['leftCrop'][0]['xCrop'] == 44) and (keypoints['leftCrop'][0]['yCrop'] == 296):
            print(idx)

In [ ]:
idx = 0
left_image_f, _, _ = s3_access_utils.download_from_url(df.left_image_url.iloc[idx])
print(df.left_image_url.iloc[idx])
right_image_f, _, _ = s3_access_utils.download_from_url(df.right_image_url.iloc[idx])
keypoints = df.keypoints.iloc[idx]


left_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['leftCrop']}
right_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['rightCrop']}
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)

# crop the data
l_width = df.left_crop_metadata.iloc[idx]['width']
l_height = df.left_crop_metadata.iloc[idx]['height']
r_width = df.right_crop_metadata.iloc[idx]['width']
r_height = df.right_crop_metadata.iloc[idx]['height']
print(l_width, l_height, r_width, r_height)
padding = 100
cropL_x_left = max(min([kp[0] for kp in left_keypoints_dict.values()]) - padding, 0)
cropL_x_right = min(max([kp[0] for kp in left_keypoints_dict.values()]) + padding, l_width)
cropL_y_top = max(min([kp[1] for kp in left_keypoints_dict.values()]) - padding, 0)
cropL_y_bottom = min(max([kp[1] for kp in left_keypoints_dict.values()]) + padding, l_height)

cropR_x_left = max(min([kp[0] for kp in right_keypoints_dict.values()]) - padding, 0)
cropR_x_right = min(max([kp[0] for kp in right_keypoints_dict.values()]) + padding, r_width)
cropR_y_top = max(min([kp[1] for kp in right_keypoints_dict.values()]) - padding, 0)
cropR_y_bottom = min(max([kp[1] for kp in right_keypoints_dict.values()]) + padding, r_height)

imageL = imageL[cropL_y_top:cropL_y_bottom, cropL_x_left:cropL_x_right]
imageR = imageR[cropR_y_top:cropR_y_bottom, cropR_x_left:cropR_x_right]

#modify keypoints
modified_keypoints = {'leftCrop': [], 'rightCrop': []}
for item in keypoints['leftCrop']:
    modified_item = copy(item)
    modified_item['xCrop'] = item['xCrop'] - cropL_x_left
    modified_item['yCrop'] = item['yCrop'] - cropL_y_top
    modified_keypoints['leftCrop'].append(modified_item)

for item in keypoints['rightCrop']:
    modified_item = copy(item)
    modified_item['xCrop'] = item['xCrop'] - cropR_x_left
    modified_item['yCrop'] = item['yCrop'] - cropR_y_top
    modified_keypoints['rightCrop'].append(modified_item)

    
    

 

In [ ]:
left_crop_url = df.left_image_url.iloc[idx]
right_crop_url = df.right_image_url.iloc[idx]
keypoints = json.dumps(df.keypoints.iloc[idx])
cm = json.dumps(df.camera_metadata.iloc[idx])
left_crop_metadata = json.dumps(df.left_crop_metadata.iloc[idx])
right_crop_metadata = json.dumps(df.right_crop_metadata.iloc[idx])
H = find_matches_and_homography_2(left_crop_url, right_crop_url, keypoints, cm, left_crop_metadata, right_crop_metadata)

In [ ]:
np.array(H).sum()

In [ ]:
# good, matchesMask, H, kp1, kp2 = find_matches_and_homography(imageL, imageR)
# adjusted_keypoints = adjust_keypoints(modified_keypoints, H)



In [ ]:
def enhance_2(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image 

In [ ]:
def find_matches_and_homography_2(left_crop_url, right_crop_url, keypoints, cm, left_crop_metadata, right_crop_metadata, MIN_MATCH_COUNT=11, GOOD_PERC=0.7, FLANN_INDEX_KDTREE=0):
    
    left_image_f, _, _ = s3_access_utils.download_from_url(df.left_image_url.iloc[idx])
    right_image_f, _, _ = s3_access_utils.download_from_url(df.right_image_url.iloc[idx])
    imageL = cv2.imread(left_image_f)
    imageR = cv2.imread(right_image_f)
    
#     imageL = load_image(left_crop_url)
#     imageR = load_image(right_crop_url)

    # crop the data

    keypoints = json.loads(keypoints)
    cm = json.loads(cm)
    left_crop_metadata = json.loads(left_crop_metadata)
    right_crop_metadata = json.loads(right_crop_metadata)
    print('Camera Metadata: {}'.format(cm))
    if 'leftCrop' in keypoints and 'rightCrop' in keypoints:
        left_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['leftCrop']}
        right_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['rightCrop']}
        print(left_keypoints_dict)
        
        # crop the data
        l_width = left_crop_metadata['width']
        l_height = left_crop_metadata['height']
        r_width = right_crop_metadata['width']
        r_height = right_crop_metadata['height']
        padding = 100
        cropL_x_left = max(min([kp[0] for kp in left_keypoints_dict.values()]) - padding, 0)
        cropL_x_right = min(max([kp[0] for kp in left_keypoints_dict.values()]) + padding, l_width)
        cropL_y_top = max(min([kp[1] for kp in left_keypoints_dict.values()]) - padding, 0)
        cropL_y_bottom = min(max([kp[1] for kp in left_keypoints_dict.values()]) + padding, l_height)

        cropR_x_left = max(min([kp[0] for kp in right_keypoints_dict.values()]) - padding, 0)
        cropR_x_right = min(max([kp[0] for kp in right_keypoints_dict.values()]) + padding, r_width)
        cropR_y_top = max(min([kp[1] for kp in right_keypoints_dict.values()]) - padding, 0)
        cropR_y_bottom = min(max([kp[1] for kp in right_keypoints_dict.values()]) + padding, r_height)
        

        imageL = imageL[cropL_y_top:cropL_y_bottom, cropL_x_left:cropL_x_right]
        imageR = imageR[cropR_y_top:cropR_y_bottom, cropR_x_left:cropR_x_right]

        sift = cv2.KAZE_create()
        img1 = enhance(imageL)
        img2 = enhance(imageR)
        kp1, des1 = sift.detectAndCompute(img1, None)
        kp2, des2 = sift.detectAndCompute(img2, None)

        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)


        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []
        H = []
        matchesMask = []
        for m,n in matches:
            if m.distance < GOOD_PERC*n.distance:
                good.append(m)
        if len(good)>=MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1, 1 ,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            matchesMask = mask.ravel().tolist()
            H = [] if H is None else H.tolist()
        else:
            print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
            matchesMask = None

        return H
    return []

In [ ]:
def plot_world_keypoints_3D(wkps):
    norm_wkps = _normalize_world_keypoints(wkps)
    body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
    xs = [norm_wkps[bp][0] for bp in body_parts]
    ys = [norm_wkps[bp][1] for bp in body_parts]
    zs = [norm_wkps[bp][2] for bp in body_parts]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim3d(0, max(xs))
    ax.set_ylim3d(-0.3, 0.3)
    ax.set_zlim3d(-0.3, 0.3)
    ax.scatter(xs, ys, zs, color='blue')

%matplotlib notebook

cm = df.camera_metadata.iloc[idx]
wkps = pixel2world(adjusted_keypoints['leftCrop'], adjusted_keypoints['rightCrop'], cm)
plot_world_keypoints_3D(wkps)

In [ ]:
%matplotlib notebook

cm = original_df.camera_metadata.iloc[idx]
adjusted_wkps = pixel2world(adjusted_keypoints['leftCrop'], adjusted_keypoints['rightCrop'], cm)
plot_world_keypoints_3D(adjusted_wkps)

In [ ]:
def coord2biomass(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)

    interaction_values_quadratic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values_quadratic.append(dist1 * dist2)

    interaction_values_cubic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            for k in range(j, len(pairwise_distances)):
                dist1 = pairwise_distances[i]
                dist2 = pairwise_distances[j]
                dist3 = pairwise_distances[k]
                interaction_values_cubic.append(dist1 * dist2 * dist3)


    X = np.array(pairwise_distances + interaction_values_quadratic + interaction_values_cubic)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction

In [ ]:
model_f = '/root/alok/repos/cv_algorithms/biomass-production/src/model.pkl'
model = pickle.load(open(model_f, 'rb'))


In [ ]:
def draw_matches_3D(img1, kp1, img2, kp2, matches, matchesMask): 
    
    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    i=0
    wkps = []
    for m in matches:
        # Generate random color for RGB/BGR and grayscale images as needed.
        
        if matchesMask[i] == 1:
            p1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
            p2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int))
            p1_x_frame = p1[0] + df.left_crop_metadata.iloc[idx]['x_coord']
            p1_y_frame = p1[1] + df.left_crop_metadata.iloc[idx]['y_coord']
            p2_x_frame = p2[0] + df.right_crop_metadata.iloc[idx]['x_coord']
            params = df.camera_metadata.iloc[idx]
#             disp = abs(p1_x_frame - p2_x_frame)
#             depth = depth_from_disp(disp, params)
#             wkp = convert_to_world_point(p1_y_frame, p1_x_frame, depth, params)
#             wkps.append(wkp)
        i += 1
        
    return wkps


In [ ]:
def generate_adjusted_weight(left_image_url, right_image_url, keypoints, cm, kpid, weight_dict):
    try:
        left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
        right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
        imageL = cv2.imread(left_image_f)
        imageR = cv2.imread(right_image_f)
        
        # crop the data
        
        left_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in keypoints['leftCrop']}
        right_keypoints_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in keypoints['rightCrop']}
        
        width = cm['pixelCountWidth']
        height = cm['pixelCountHeight']
        padding = 100
        cropL_x_left = max(min([kp[0] for kp in left_keypoints_dict.values()]) - padding, 0)
        cropL_x_right = min(max([kp[0] for kp in left_keypoints_dict.values()]) + padding, width)
        cropL_y_top = max(min([kp[1] for kp in left_keypoints_dict.values()]) - padding, 0)
        cropL_y_bottom = min(max([kp[1] for kp in left_keypoints_dict.values()]) + padding, height)

        cropR_x_left = max(min([kp[0] for kp in right_keypoints_dict.values()]) - padding, 0)
        cropR_x_right = min(max([kp[0] for kp in right_keypoints_dict.values()]) + padding, width)
        cropR_y_top = max(min([kp[1] for kp in right_keypoints_dict.values()]) - padding, 0)
        cropR_y_bottom = min(max([kp[1] for kp in right_keypoints_dict.values()]) + padding, height)

        imageL = imageL[cropL_y_top:cropL_y_bottom, cropL_x_left:cropL_x_right]
        imageR = imageR[cropR_y_top:cropR_y_bottom, cropR_x_left:cropR_x_right]

        #modify keypoints
        modified_keypoints = {'leftCrop': [], 'rightCrop': []}
        for item in keypoints['leftCrop']:
            modified_item = copy(item)
            modified_item['xCrop'] = item['xCrop'] - cropL_x_left
            modified_item['yCrop'] = item['yCrop'] - cropL_y_top
            modified_keypoints['leftCrop'].append(modified_item)

        for item in keypoints['rightCrop']:
            modified_item = copy(item)
            modified_item['xCrop'] = item['xCrop'] - cropR_x_left
            modified_item['yCrop'] = item['yCrop'] - cropR_y_top
            modified_keypoints['rightCrop'].append(modified_item)
        
        good, matchesMask, H, kp1, kp2 = find_matches_and_homography(imageL, imageR)
        adjusted_keypoints = adjust_keypoints(modified_keypoints, H)
        adjusted_wkps = pixel2world(adjusted_keypoints['leftCrop'], 
                                    adjusted_keypoints['rightCrop'],
                                    cm)
        
        two_dimensional_locs = []
        i=0
        for m in good:
            if matchesMask[i] == 1:
                p1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
                p2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int))
                p1_x_frame = p1[0] + cropL_x_left
                p1_y_frame = p1[1] + cropL_y_top
                p2_x_frame = p2[0] + cropR_x_left
                disp = abs(p1_x_frame - p2_x_frame)
                two_dimensional_locs.append([p1_x_frame, p1_y_frame, disp])
            i += 1
        
        
        item_to_add = {
            'adjusted_wkps': adjusted_wkps,
            'two_dimensional_locs': two_dimensional_locs,
        }
        
        weight_dict[kpid] = item_to_add
        
    except Exception as e:
        print('Error: {}'.format(e))
        
    print(len(weight_dict.keys()))



In [ ]:
manager = Manager()
weight_dict = manager.dict()

args = []
for idx, row in df.iterrows():
    args.append((row.left_image_url, row.right_image_url, row.keypoints, 
                 row.camera_metadata, row.id, weight_dict))

pool = Pool(processes=10)
pool.starmap(generate_adjusted_weight, args)


In [ ]:
left_crop_metadata

In [ ]:
for k, v in weight_dict.items():
    print(v.keys())
    break
    

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
weights = np.array(weight_dict.values())
mask = (weights > 0) & (weights < 20000)
plt.hist(weights[mask], bins=20)
plt.grid()
plt.show()

In [ ]:
print(np.mean(weights[mask]))

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
weights = original_df.estimated_biomass_g.values
mask = (weights > 0) & (weights < 20000)
plt.hist(weights[mask], bins=20)
plt.grid()
plt.show()

In [ ]:
print(np.mean(weights[mask]))

In [ ]:
coord2biomass(wkps, model)

In [ ]:
coord2biomass(adjusted_wkps, model)

In [ ]:
tdf = pd.read_csv('/root/data/temp/imr_austevoll_data.csv')

In [ ]:
tdf.index = pd.to_datetime(tdf.captured_at)

In [ ]:
tdf.annotation.resample('H', how=len)

In [ ]:
tdf.to_csv('/root/data/temp/imr_austevoll_hourly_breakdown.csv')

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()